In [48]:
%load_ext autoreload
%autoreload 2
from src.models.vae import *
from src.data.data_loader import get_loader, TwitterDataset

from src.models.common import (
    EmbeddingPacked,
    get_numpy,
    get_variable,
    simple_elementwise_apply,
    cuda,
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
enc = Encoder()

In [50]:
data = pd.read_pickle("../data/interim/hydrated/200316.pkl")
train = TwitterDataset(data.iloc[0:1000, :])
train_loader = get_loader(train, 10)

In [51]:
x = next(iter(train_loader))
x = get_variable(x)

# Forward pass
output = enc(x)